# NY市場の株価データ解析

daru, statsample, Nyaplot, IRubyを使って、株式市場のブラウン運動を観察する。
収益率の対数が正規分布に従うかどうか。

In [1]:
#$".push("backports.rb") # Ruby 2.xでbackportsとprimeがrequireされるとエラーになるのを避けるため
#require "backports/tools"
#require "backports/version"
#require "backports/rails"
require "daru"
require "statsample"

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

In [2]:
# ニューヨーク市場の銘柄から100のサンプル抽出
# stock_ticker = Daru::DataFrame.from_csv("stock_ticker.csv")
# sample_ticker = stock_ticker.where(stock_ticker["Exchange"].eq("NYQ"))["Ticker"].to_a.shuffle.take(100)
# Daru::DataFrame.new("Ticker" => sample_ticker).write_csv("sample_ticker.csv")

## 株価データのダウンロード

* stock_ticker.csvをもとに、pandasのget_data_yahoo()を使って2006年1月1日以降の株価データを[ダウンロード](StockDownload.ipynb)
* 証券コードに'-'が含まれる4件はエラー

In [3]:
# pandasでダウンロードしたCSVをロード
# 最初の日付が2006-01-03のものだけ抽出
stocks = Dir["stock/*.csv"].map { |file|
  name = file.slice(/([^\/]+)\.csv\z/, 1)
  Daru::DataFrame.from_csv(file, name: name)
}.select {|stock| stock["Date"][0] == "2006-01-03" }
stocks.size

61

## 株価の分布

In [4]:
# 直近1か月の終値の分布
recent_close = Daru::DataFrame.new(
  close: stocks.flat_map { |stock|
    stock["Adj Close"][-30..-1].to_a
  }
)
recent_close.plot type: :histogram, x: :close do |plot, diagram|
  diagram.bin_num(100)
  plot.yrange([0, 200])
  plot.x_label("P($)")
  plot.y_label("No per P")
end

#<Nyaplot::Frame:0x007f629669a820 @properties={:panes=>[#<Nyaplot::Plot:0x007f6294d1cdd0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f629669b040 @properties={:type=>:histogram, :options=>{:value=>:close, :bin_num=>100}, :data=>"694880b1-4d81-4788-a309-50a52a1146ca"}, @xrange=[0, 200.5], @yrange=[0, 1830]>], :options=>{:yrange=>[0, 200], :x_label=>"P($)", :y_label=>"No per P", :width=>700, :xrange=>[0, 200.5]}}>], :data=>{"694880b1-4d81-4788-a309-50a52a1146ca"=>#<Nyaplot::DataFrame:0x007f62966f36f0 @name="694880b1-4d81-4788-a309-50a52a1146ca", @rows=[{:close=>0.9}, {:close=>0.92}, {:close=>0.96}, {:close=>0.93}, {:close=>0.85}, {:close=>0.87}, {:close=>0.9}, {:close=>0.86}, {:close=>0.84}, {:close=>0.78}, {:close=>0.73}, {:close=>0.78}, {:close=>0.77}, {:close=>0.74}, {:close=>0.73}, {:close=>0.72}, {:close=>0.7}, {:close=>0.67}, {:close=>0.72}, {:close=>0.73}, {:close=>0.67}, {:close=>0.55}, {:close=>0.5}, {:close=>0.5}, {:close=>0.48}, {:close=>0.58}, {:close=>0.57}, {:close=>0.53}, {:close=>0.52}, {:close=>0.48}, {:close=>41.151256}, {:close=>41.031834}, {:close=>40.942267}, {:close=>41.519481}, {:close=>41.081593}, {:close=>41.021884}, {:close=>39.777892}, {:close=>39.628611}, {:close=>38.633418}, {:close=>37.658128000000005}, {:close=>36.971446}, {:close=>37.240145}, {:close=>37.538706}, {:close=>36.28476}, {:close=>36.891829}, {:close=>35.608028000000004}, {:close=>35.757309}, {:close=>35.29952}, {:close=>34.145096}, {:close=>34.981059}, {:close=>34.503362}, {:close=>35.389088}, {:close=>35.259712}, {:close=>35.199999}, {:close=>36.220001}, {:close=>35.5}, {:close=>34.290001000000004}, {:close=>34.34}, {:close=>34.950001}, {:close=>34.459998999999996}, {:close=>14.009310000000001}, {:close=>14.02921}, {:close=>14.049109}, {:close=>14.059059}, {:close=>14.168505999999999}, {:close=>14.258054000000001}, {:close=>14.307804}, {:close=>14.357553}, {:close=>14.496849}, {:close=>14.437151}, {:close=>14.467}, {:close=>14.407301}, {:close=>14.467}, {:close=>14.34}, {:close=>14.4}, {:close=>14.43}, {:close=>14.4}, {:close=>14.29}, {:close=>14.33}, {:close=>14.45}, {:close=>14.42}, {:close=>14.44}, {:close=>14.41}, {:close=>14.48}, {:close=>14.62}, {:close=>14.67}, {:close=>14.63}, {:close=>14.7}, {:close=>14.76}, {:close=>14.74}, {:close=>9.52}, {:close=>9.44}, {:close=>9.16}, {:close=>9.42}, {:close=>9.44}, {:close=>9.33}, {:close=>9.75}, {:close=>9.5}, {:close=>9.26}, {:close=>8.72}, {:close=>8.23}, {:close=>7.85}, {:close=>7.79}, {:close=>8.15}, {:close=>8.48}, {:close=>8.01}, {:close=>8.21}, {:close=>8.24}, {:close=>8.21}, {:close=>8.36}, {:close=>8.13}, {:close=>8.78}, {:close=>9.4}, {:close=>8.66}, {:close=>9.3}, {:close=>8.97}, {:close=>8.51}, {:close=>9.61}, {:close=>10.03}, {:close=>9.86}, {:close=>200.5}, {:close=>199.41000400000001}, {:close=>200.270004}, {:close=>200.320007}, {:close=>198.740005}, {:close=>197.229996}, {:close=>194.669998}, {:close=>195.5}, {:close=>193.100006}, {:close=>187.809998}, {:close=>182.389999}, {:close=>163.550003}, {:close=>164.009995}, {:close=>157.83999599999999}, {:close=>164.779999}, {:close=>165.0}, {:close=>169.440002}, {:close=>170.0}, {:close=>170.449997}, {:close=>170.179993}, {:close=>170.169998}, {:close=>170.990005}, {:close=>166.919998}, {:close=>160.119995}, {:close=>160.979996}, {:close=>159.270004}, {:close=>155.0}, {:close=>152.369995}, {:close=>153.809998}, {:close=>154.360001}, {:close=>21.709999}, {:close=>21.77}, {:close=>21.790001}, {:close=>21.99}, {:close=>21.74}, {:close=>21.459999}, {:close=>20.860001}, {:close=>21.74}, {:close=>21.860001}, {:close=>21.17}, {:close=>20.77}, {:close=>20.84}, {:close=>20.639999}, {:close=>20.280001000000002}, {:close=>20.32}, {:close=>19.92}, {:close=>19.83}, {:close=>19.629998999999998}, {:close=>20.07}, {:close=>20.4}, {:close=>20.01}, {:close=>20.610001}, {:close=>20.030001000000002}, {:close=>20.16}, {:close=>20.719998999999998}, {:close=>20.690001000000002}, {:close=>20.6}, {:close=>20.469998999999998}, {:close=>20.280001000

In [5]:
# 直近1か月の終値の対数の分布
recent_log_close = Daru::DataFrame.new(
  close: stocks.flat_map { |stock|
    stock["Adj Close"].tail(30).map { |x| Math.log(x) }
  }
)
recent_log_close.plot type: :histogram, x: :close do |plot, diagram|
  diagram.bin_num(100)
  plot.yrange([0, 100])
  plot.x_label("ln(P)")
  plot.y_label("No per ln(P)")
end

#<Nyaplot::Frame:0x007f62969844f0 @properties={:panes=>[#<Nyaplot::Plot:0x007f6297599b00 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f6296984ce8 @properties={:type=>:histogram, :options=>{:value=>:close, :bin_num=>100}, :data=>"7e3074be-9d85-40dc-b5a5-180d518209b9"}, @xrange=[-0.7339691750802004, 5.300814246746624], @yrange=[0, 1830]>], :options=>{:yrange=>[0, 100], :x_label=>"ln(P)", :y_label=>"No per ln(P)", :width=>700, :xrange=>[-0.7339691750802004, 5.300814246746624]}}>], :data=>{"7e3074be-9d85-40dc-b5a5-180d518209b9"=>#<Nyaplot::DataFrame:0x007f62969d78a8 @name="7e3074be-9d85-40dc-b5a5-180d518209b9", @rows=[{:close=>-0.10536051565782628}, {:close=>-0.08338160893905101}, {:close=>-0.040821994520255166}, {:close=>-0.07257069283483537}, {:close=>-0.16251892949777494}, {:close=>-0.13926206733350766}, {:close=>-0.10536051565782628}, {:close=>-0.15082288973458366}, {:close=>-0.1743533871447778}, {:close=>-0.2484613592984996}, {:close=>-0.31471074483970024}, {:close=>-0.2484613592984996}, {:close=>-0.2613647641344075}, {:close=>-0.3011050927839216}, {:close=>-0.31471074483970024}, {:close=>-0.3285040669720361}, {:close=>-0.35667494393873245}, {:close=>-0.40047756659712525}, {:close=>-0.3285040669720361}, {:close=>-0.31471074483970024}, {:close=>-0.40047756659712525}, {:close=>-0.5978370007556204}, {:close=>-0.6931471805599453}, {:close=>-0.6931471805599453}, {:close=>-0.7339691750802004}, {:close=>-0.5447271754416722}, {:close=>-0.5621189181535413}, {:close=>-0.6348782724359695}, {:close=>-0.6539264674066639}, {:close=>-0.7339691750802004}, {:close=>3.7172544491360138}, {:close=>3.7143482044558556}, {:close=>3.7121629524177115}, {:close=>3.7261627387795073}, {:close=>3.715560162302668}, {:close=>3.7141056804047508}, {:close=>3.6833112805687382}, {:close=>3.6795513573829584}, {:close=>3.65411765321963}, {:close=>3.6285488140806224}, {:close=>3.6101458849778125}, {:close=>3.6173873460640142}, {:close=>3.6253725606654874}, {:close=>3.5913978184047943}, {:close=>3.607990090211478}, {:close=>3.5725711180533923}, {:close=>3.5767546957959113}, {:close=>3.56386936611309}, {:close=>3.530618973948125}, {:close=>3.554806743574733}, {:close=>3.5410567685648044}, {:close=>3.566403524033797}, {:close=>3.5627410090407343}, {:close=>3.5610460541949602}, {:close=>3.5896114800643226}, {:close=>3.56953269648137}, {:close=>3.53485379563785}, {:close=>3.5363108554693294}, {:close=>3.553918497292122}, {:close=>3.5397992020806304}, {:close=>2.6397221086007363}, {:close=>2.641141584617402}, {:close=>2.642558977398745}, {:close=>2.6432669566890072}, {:close=>2.651021614128507}, {:close=>2.657321940035719}, {:close=>2.6608051226728073}, {:close=>2.6642761451999624}, {:close=>2.673931315466387}, {:close=>2.669804814829664}, {:close=>2.6718701936474534}, {:close=>2.6677350919827303}, {:close=>2.6718701936474534}, {:close=>2.663052835171474}, {:close=>2.667228206581955}, {:close=>2.6693093727857793}, {:close=>2.667228206581955}, {:close=>2.659559991941776}, {:close=>2.6623552418400807}, {:close=>2.670694414558441}, {:close=>2.668616131856803}, {:close=>2.67000213346468}, {:close=>2.667922410011431}, {:close=>2.6727683869575705}, {:close=>2.6823904543216326}, {:close=>2.6858045921548905}, {:close=>2.683074215032033}, {:close=>2.6878474937846906}, {:close=>2.6919208191723265}, {:close=>2.6905648867611904}, {:close=>2.253394848803274}, {:close=>2.244955980157409}, {:close=>2.214846178686039}, {:close=>2.2428350885882717}, {:close=>2.244955980157409}, {:close=>2.2332350148592526}, {:close=>2.277267285009756}, {:close=>2.2512917986064953}, {:close=>2.225704048658088}, {:close=>2.1656192379208883}, {:close=>2.1077860146889784}, {:close=>2.060513531794317}, {:close=>2.052840859882657}, {:close=>2.0980179272527715}, {:close=>2.1377104498038118}, {:close=>2.080690761080268}, {:close=>2.105352923464337}, {:close=>2.1090003439213802}, {:close=>2.105352923464337}, {:close=>2.1234584270966104}, {:close=>2.0955609235597192}, {:close=>2.172476407647025}, {:close=>2.

In [6]:
# 最初の銘柄の統計量
stocks[0].describe

,Adj Close,Close,High,Low,Open,Volume
count,2541,2541,2541,2541,2541,2541
mean,7.613455325462419,7.613455325462419,7.7849822861078,7.4414600413223235,7.622892564344714,449029.3978748524
std,6.901125375886355,6.901125375886355,7.0004775074128105,6.79228296852532,6.905777926206229,455525.4043124349
min,0.48,0.48,0.5,0.46,0.5,36000
max,26.5,26.5,26.76,26.129998999999998,26.450001,9079800


## 対数収益率の分布

In [7]:
# 日次収益率の対数（log(P(t + 1day) / P0(t))）を計算
def get_log_return(stock)
  Daru::DataFrame.new({
    date: stock["Date"].drop(1),
    volume: stock["Volume"].drop(1),
    # [a, b, c, d, ...].each_cons(2) #=> [a, b], [b, c], [c, d], ...
    log_return: stock["Adj Close"].each_cons(2).map { |prev, current| Math.log(current / prev) }
  }, name: stock.name, order: [:date, :volume, :log_return])
end

# 遅い例
def get_log_return_slow(stock)
  Daru::DataFrame.new(
    # 行毎の処理は遅い
    stock.each_row.each_cons(2).map { |prev, current|
      {
        date: current["Date"],
        volume: current["Volume"],
        log_return: Math.log(current["Adj Close"] / prev["Adj Close"])
      }
    }
  )
end

log_returns = stocks.map { |stock|
  get_log_return(stock)
}
nil

In [8]:
# 最初の3銘柄のヒストグラムをプロット
log_returns.take(3).each do |log_return|
  log_return.plot type: :histogram, x: :log_return do |plot, diagram|
    diagram.bin_num(150)
    plot.yrange([0, 520])
    plot.x_label("ln[P(t + 1d)/P(t)] of #{log_return.name}")
    plot.y_label("No per ln[P(t + 1d)/P(t)]")
  end
end
nil

#<Nyaplot::Frame:0x007f62965637e0 @properties={:panes=>[#<Nyaplot::Plot:0x007f6293398a58 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f6296568010 @properties={:type=>:histogram, :options=>{:value=>:log_return, :bin_num=>150}, :data=>"96cae89a-f04f-49f7-aee8-c7ef787ab19d"}, @xrange=[-0.2546422183735808, 0.23502078879480565], @yrange=[0, 2540]>], :options=>{:yrange=>[0, 520], :x_label=>"ln[P(t + 1d)/P(t)] of CVO", :y_label=>"No per ln[P(t + 1d)/P(t)]", :width=>700, :xrange=>[-0.2546422183735808, 0.23502078879480565]}}>], :data=>{"96cae89a-f04f-49f7-aee8-c7ef787ab19d"=>#<Nyaplot::DataFrame:0x007f62966ce3a0 @name="96cae89a-f04f-49f7-aee8-c7ef787ab19d", @rows=[{:date=>"2006-01-04", :volume=>365900, :log_return=>0.009430610335608217}, {:date=>"2006-01-05", :volume=>90300, :log_return=>-0.013081581897489276}, {:date=>"2006-01-06", :volume=>642000, :log_return=>0.03097114664826428}, {:date=>"2006-01-09", :volume=>283900, :log_return=>0.000708968480599987}, {:date=>"2006-01-10", :volume=>575500, :log_return=>0.03001970624065062}, {:date=>"2006-01-11", :volume=>177300, :log_return=>-0.013850636933898937}, {:date=>"2006-01-12", :volume=>146200, :log_return=>-0.014045174703047527}, {:date=>"2006-01-13", :volume=>169500, :log_return=>0.03543267499464958}, {:date=>"2006-01-17", :volume=>348200, :log_return=>-0.04181793968332148}, {:date=>"2006-01-18", :volume=>217500, :log_return=>-0.018678703974544426}, {:date=>"2006-01-19", :volume=>197600, :log_return=>-0.007278052504852913}, {:date=>"2006-01-20", :volume=>538900, :log_return=>-0.03342308879149533}, {:date=>"2006-01-23", :volume=>207200, :log_return=>0.008273834337396398}, {:date=>"2006-01-24", :volume=>398300, :log_return=>-0.006764400088542037}, {:date=>"2006-01-25", :volume=>276800, :log_return=>0.012739025777429712}, {:date=>"2006-01-26", :volume=>187800, :log_return=>0.04797431537544265}, {:date=>"2006-01-27", :volume=>266500, :log_return=>-0.0021314395280699863}, {:date=>"2006-01-30", :volume=>313700, :log_return=>0.0021314395280700397}, {:date=>"2006-01-31", :volume=>287600, :log_return=>0.0077766386966260995}, {:date=>"2006-02-01", :volume=>527200, :log_return=>0.0007039775017943326}, {:date=>"2006-02-02", :volume=>347900, :log_return=>-0.0177498447747828}, {:date=>"2006-02-03", :volume=>186700, :log_return=>0.0}, {:date=>"2006-02-06", :volume=>275800, :log_return=>0.007848773792071204}, {:date=>"2006-02-07", :volume=>287300, :log_return=>-0.015759638492950204}, {:date=>"2006-02-08", :volume=>115500, :log_return=>0.01433716314640725}, {:date=>"2006-02-09", :volume=>234800, :log_return=>-0.006426298445528293}, {:date=>"2006-02-10", :volume=>285100, :log_return=>0.002861232281032195}, {:date=>"2006-02-13", :volume=>154800, :log_return=>0.0035650661644961446}, {:date=>"2006-02-14", :volume=>1237900, :log_return=>0.0077986924858188394}, {:date=>"2006-02-15", :volume=>221900, :log_return=>0.009138903676202348}, {:date=>"2006-02-16", :volume=>181100, :log_return=>0.01182622478884263}, {:date=>"2006-02-17", :volume=>246000, :log_return=>0.005517255374754659}, {:date=>"2006-02-21", :volume=>194600, :log_return=>0.006170742926659674}, {:date=>"2006-02-22", :volume=>182500, :log_return=>-0.003423488451520917}, {:date=>"2006-02-23", :volume=>122400, :log_return=>-0.013812374287611359}, {:date=>"2006-02-24", :volume=>89800, :log_return=>0.0006951686063757036}, {:date=>"2006-02-27", :volume=>85100, :log_return=>-0.011181108797515485}, {:date=>"2006-02-28", :volume=>216400, :log_return=>-0.009887086191172244}, {:date=>"2006-03-01", :volume=>339300, :log_return=>0.032125667706912775}, {:date=>"2006-03-02", :volume=>1107600, :log_return=>0.02242703578744421}, {:date=>"2006-03-03", :volume=>345600, :log_return=>0.004693270431749567}, {:date=>"2006-03-06", :volume=>307500, :log_return=>-0.02095379130352983}, {:date=>"2006-03-07", :volume=>639500, :log_return=>0.03424302342221266}, {:date=>"2006-03-08", :volume=>310800, :log_return=>0.0026367846523031938}, {:date=>"2006-03-09", :volume=>363200,

#<Nyaplot::Frame:0x007f6296520da0 @properties={:panes=>[#<Nyaplot::Plot:0x007f629501b298 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f62965216b0 @properties={:type=>:histogram, :options=>{:value=>:log_return, :bin_num=>150}, :data=>"45e71d49-a191-4cb0-9a7d-494486a038a6"}, @xrange=[-0.3168744682392347, 0.22159435209794193], @yrange=[0, 2540]>], :options=>{:yrange=>[0, 520], :x_label=>"ln[P(t + 1d)/P(t)] of BK", :y_label=>"No per ln[P(t + 1d)/P(t)]", :width=>700, :xrange=>[-0.3168744682392347, 0.22159435209794193]}}>], :data=>{"45e71d49-a191-4cb0-9a7d-494486a038a6"=>#<Nyaplot::DataFrame:0x007f6294d86640 @name="45e71d49-a191-4cb0-9a7d-494486a038a6", @rows=[{:date=>"2006-01-04", :volume=>3071200, :log_return=>0.004909456575330774}, {:date=>"2006-01-05", :volume=>2514200, :log_return=>0.004885577924340135}, {:date=>"2006-01-06", :volume=>3920300, :log_return=>0.013012770458041365}, {:date=>"2006-01-09", :volume=>3089700, :log_return=>0.005696339614212328}, {:date=>"2006-01-10", :volume=>2168400, :log_return=>-0.0017952804794038875}, {:date=>"2006-01-11", :volume=>2757400, :log_return=>0.010427614986358103}, {:date=>"2006-01-12", :volume=>2810600, :log_return=>-0.008931376420886922}, {:date=>"2006-01-13", :volume=>2801200, :log_return=>-0.0032948876223049565}, {:date=>"2006-01-17", :volume=>2563400, :log_return=>-0.011770195634285163}, {:date=>"2006-01-18", :volume=>4594300, :log_return=>-0.01900154819135853}, {:date=>"2006-01-19", :volume=>5234300, :log_return=>-0.018423402710565483}, {:date=>"2006-01-20", :volume=>6124100, :log_return=>-0.008227912943932197}, {:date=>"2006-01-23", :volume=>3578300, :log_return=>0.0012788182081390081}, {:date=>"2006-01-24", :volume=>2648500, :log_return=>0.003189780068425619}, {:date=>"2006-01-25", :volume=>2443200, :log_return=>0.004448682033488433}, {:date=>"2006-01-26", :volume=>2843500, :log_return=>0.006636134031936442}, {:date=>"2006-01-27", :volume=>2689800, :log_return=>-0.005052035091319135}, {:date=>"2006-01-30", :volume=>2551300, :log_return=>0.002844879941739692}, {:date=>"2006-01-31", :volume=>2639200, :log_return=>0.004095221111661811}, {:date=>"2006-02-01", :volume=>2258700, :log_return=>0.0018840723618869055}, {:date=>"2006-02-02", :volume=>4236900, :log_return=>0.007502468456595339}, {:date=>"2006-02-03", :volume=>3498600, :log_return=>0.004350529533151804}, {:date=>"2006-02-06", :volume=>2256300, :log_return=>-0.0006204235198416648}, {:date=>"2006-02-07", :volume=>2774900, :log_return=>0.000310277814161333}, {:date=>"2006-02-08", :volume=>2463900, :log_return=>0.00803219207548322}, {:date=>"2006-02-09", :volume=>1658100, :log_return=>-0.0009234918544565431}, {:date=>"2006-02-10", :volume=>2944300, :log_return=>0.014371185331812147}, {:date=>"2006-02-13", :volume=>3532600, :log_return=>0.011470206155610367}, {:date=>"2006-02-14", :volume=>4222500, :log_return=>0.005089052238451033}, {:date=>"2006-02-15", :volume=>4209000, :log_return=>0.010987513196242913}, {:date=>"2006-02-16", :volume=>3620600, :log_return=>0.009991030556454896}, {:date=>"2006-02-17", :volume=>4672400, :log_return=>0.011339008037589466}, {:date=>"2006-02-21", :volume=>3185300, :log_return=>-0.008127789538164417}, {:date=>"2006-02-22", :volume=>3183000, :log_return=>0.016476774133596862}, {:date=>"2006-02-23", :volume=>3751300, :log_return=>-0.011244167371129888}, {:date=>"2006-02-24", :volume=>2731800, :log_return=>-0.0026129313000415842}, {:date=>"2006-02-27", :volume=>2011100, :log_return=>0.0063750302400890895}, {:date=>"2006-02-28", :volume=>3494400, :log_return=>-0.011037008778763402}, {:date=>"2006-03-01", :volume=>4403400, :log_return=>0.009013011768315132}, {:date=>"2006-03-02", :volume=>8062700, :log_return=>0.0054842323111011625}, {:date=>"2006-03-03", :volume=>6439200, :log_return=>-0.010416766785170069}, {:date=>"2006-03-06", :volume=>2012000, :log_return=>-0.00495698502340355}, {:date=>"2006-03-07", :volume=>1798000, :log_return=>0.0011685738728827806}, {:date=>"2006-03-08", :volume=>2484500, :l

#<Nyaplot::Frame:0x007f6296949058 @properties={:panes=>[#<Nyaplot::Plot:0x007f6296e8b138 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f62969498a0 @properties={:type=>:histogram, :options=>{:value=>:log_return, :bin_num=>150}, :data=>"1ad9b22a-aa71-48ad-8ae9-7f5c2fd4bd5d"}, @xrange=[-0.13503625982529693, 0.17908555905799614], @yrange=[0, 2540]>], :options=>{:yrange=>[0, 520], :x_label=>"ln[P(t + 1d)/P(t)] of NIO", :y_label=>"No per ln[P(t + 1d)/P(t)]", :width=>700, :xrange=>[-0.13503625982529693, 0.17908555905799614]}}>], :data=>{"1ad9b22a-aa71-48ad-8ae9-7f5c2fd4bd5d"=>#<Nyaplot::DataFrame:0x007f62969a7298 @name="1ad9b22a-aa71-48ad-8ae9-7f5c2fd4bd5d", @rows=[{:date=>"2006-01-04", :volume=>62400, :log_return=>0.008119125105935819}, {:date=>"2006-01-05", :volume=>119000, :log_return=>0.0033635582336744483}, {:date=>"2006-01-06", :volume=>79300, :log_return=>0.007360365503442475}, {:date=>"2006-01-09", :volume=>78200, :log_return=>-0.0026701882732959796}, {:date=>"2006-01-10", :volume=>113800, :log_return=>-0.010752874747146285}, {:date=>"2006-01-11", :volume=>69600, :log_return=>0.006833856008825268}, {:date=>"2006-01-12", :volume=>84500, :log_return=>0.0073900405819672405}, {:date=>"2006-01-13", :volume=>51300, :log_return=>-0.002010099781228616}, {:date=>"2006-01-17", :volume=>80100, :log_return=>0.0006705228261809023}, {:date=>"2006-01-18", :volume=>61400, :log_return=>0.007345625731470118}, {:date=>"2006-01-19", :volume=>117400, :log_return=>0.0019940669242443547}, {:date=>"2006-01-20", :volume=>55600, :log_return=>0.005297993819095053}, {:date=>"2006-01-23", :volume=>91300, :log_return=>0.0}, {:date=>"2006-01-24", :volume=>125000, :log_return=>0.005926823957264075}, {:date=>"2006-01-25", :volume=>85500, :log_return=>-0.001314052884540807}, {:date=>"2006-01-26", :volume=>86900, :log_return=>-0.007258259394060735}, {:date=>"2006-01-27", :volume=>100200, :log_return=>0.0013234990314996008}, {:date=>"2006-01-30", :volume=>73600, :log_return=>0.0}, {:date=>"2006-01-31", :volume=>60400, :log_return=>0.007905252235719723}, {:date=>"2006-02-01", :volume=>90900, :log_return=>0.0}, {:date=>"2006-02-02", :volume=>60500, :log_return=>-0.002628106747000946}, {:date=>"2006-02-03", :volume=>70200, :log_return=>-0.003294936065202748}, {:date=>"2006-02-06", :volume=>89700, :log_return=>-0.00198220942351611}, {:date=>"2006-02-07", :volume=>80500, :log_return=>-0.006635685282283574}, {:date=>"2006-02-08", :volume=>75800, :log_return=>0.011912830771002294}, {:date=>"2006-02-09", :volume=>44800, :log_return=>-0.0006581670179376904}, {:date=>"2006-02-10", :volume=>62900, :log_return=>-0.0019768648781644635}, {:date=>"2006-02-13", :volume=>46900, :log_return=>0.0007286915895936296}, {:date=>"2006-02-14", :volume=>50600, :log_return=>0.0013236326234281424}, {:date=>"2006-02-15", :volume=>68800, :log_return=>0.0006612195468242672}, {:date=>"2006-02-16", :volume=>89100, :log_return=>-0.00198485217025227}, {:date=>"2006-02-17", :volume=>58700, :log_return=>0.002645515559222554}, {:date=>"2006-02-21", :volume=>91100, :log_return=>0.0013201378678209224}, {:date=>"2006-02-22", :volume=>106900, :log_return=>-0.0006599106668063089}, {:date=>"2006-02-23", :volume=>88400, :log_return=>0.0}, {:date=>"2006-02-24", :volume=>62500, :log_return=>0.005266657797243009}, {:date=>"2006-02-27", :volume=>87600, :log_return=>0.0052390653693206255}, {:date=>"2006-02-28", :volume=>82100, :log_return=>-0.001307195358143059}, {:date=>"2006-03-01", :volume=>132400, :log_return=>0.004567668653911548}, {:date=>"2006-03-02", :volume=>79100, :log_return=>-0.01047122687643286}, {:date=>"2006-03-03", :volume=>80300, :log_return=>0.0006576218866173356}, {:date=>"2006-03-06", :volume=>49500, :log_return=>-0.004612818673531189}, {:date=>"2006-03-07", :volume=>124500, :log_return=>-0.012628942350407714}, {:date=>"2006-03-08", :volume=>102500, :log_return=>0.004671426571074886}, {:date=>"2006-03-09", :volume=>60700, :log_return=>0.0026595872086196126}, {:date=>"2006-03-10", :volume

In [9]:
# 標本の個々の値から計算した経験累積分布と、標本平均・標本標準偏差から
# 計算した正規累積分布をプロットして比較

def empirical_cumulative_distribution(v)
  x = []
  count = []
  # 簡便さのためsortを使う
  v.sort.each do |i|
    if x.empty? || x[-1] < i
      # xが空またはxの最後の要素がiより小さい時
      x.push(i)
      count.push((count[-1] || 0) + 1) # 最後のcount + 1を追加
    else
      # xの最後の値とiが同じ時
      count[-1] += 1 # 最後のcountをincrement
    end
  end
  # countの値をvの総数で割った値をy (0〜1)とする
  y = count.map { |i| i / v.count.to_f }
  Daru::DataFrame.new(x: x, y: y)
end

def normal_cumulative_distribution(v)
  x = v.min.step(v.max, (v.max - v.min) / 99).to_a
  y = x.map { |i|
    Distribution::Normal.cdf((i - v.mean) / v.std)
  }
  Daru::DataFrame.new(x: x, y: y)
end

def plot_cumulative_distribution(v, x_label)
  ecd = empirical_cumulative_distribution(v)
  ncd = normal_cumulative_distribution(v)
  plot = Nyaplot::Plot.new
  colors = Nyaplot::Colors.qual
  plot.add_with_df(ecd, :line, :x, :y).configure do |d|
    d.title("Empirical")
  end
  plot.add_with_df(ncd, :line, :x, :y).configure do |d|
    d.title("Normal")
    d.color("#44bb44")
  end
  plot.xrange([-0.1, 0.1])
  plot.x_label(x_label)
  plot.y_label("Cumulative probability")
  plot.legend(true)
  plot.show
end

log_returns.take(3).each do |log_return|
  plot_cumulative_distribution(log_return[:log_return], "ln[P(t + 1d)/P(t)] of #{log_return.name}")
end
nil

#<Nyaplot::Frame:0x007f629537abc8 @properties={:panes=>[#<Nyaplot::Plot:0x007f629538a4d8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f6295389470 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :title=>"Empirical"}, :data=>"a78f922c-509f-4a87-8716-2bfa43485327"}, @xrange=[-0.2546422183735808, 0.23502078879480565], @yrange=[0.0003937007874015748, 1.0]>, #<Nyaplot::Diagram:0x007f62953880e8 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :title=>"Normal", :color=>"#44bb44"}, :data=>"8ffa54f4-24f4-43fe-89f8-af5f386557d1"}, @xrange=[-0.2546422183735808, 0.23502078879480565], @yrange=[1.278015471228855e-10, 0.9999999981787691]>], :options=>{:xrange=>[-0.1, 0.1], :x_label=>"ln[P(t + 1d)/P(t)] of CVO", :y_label=>"Cumulative probability", :legend=>true, :zoom=>true, :width=>800, :yrange=>[1.278015471228855e-10, 1.0]}}>], :data=>{"a78f922c-509f-4a87-8716-2bfa43485327"=>
#<Daru::DataFrame:70030690223140 @name = a78f922c-509f-4a87-8716-2bfa43485327 @size = 2225>
                    x          y 
         0 -0.2546422 0.00039370 
         1 -0.2358570 0.00078740 
         2 -0.2266057 0.00118110 
         3 -0.2100403 0.00157480 
         4 -0.1973594 0.00196850 
         5 -0.1727126 0.00236220 
         6 -0.1687871 0.00275590 
         7 -0.1624029 0.00314960 
         8 -0.1459539 0.00354330 
         9 -0.1451820 0.00393700 
        10 -0.1415160 0.00433070 
        11 -0.1347325 0.00472440 
        12 -0.1342353 0.00511811 
        13 -0.1276155 0.00551181 
        14 -0.1256086 0.00590551 
       ...        ...        ... 
, "8ffa54f4-24f4-43fe-89f8-af5f386557d1"=>
#<Daru::DataFrame:70030693717600 @name = 8ffa54f4-24f4-43fe-89f8-af5f386557d1 @size = 100>
                    x          y 
         0 -0.2546422 1.27801547 
         1 -0.2496961 2.82146639 
         2 -0.2447500 6.13682382 
         3 -0.2398039 1.31506561 
         4 -0.2348578 2.77646183 
         5 -0.2299117 5.77540681 
         6 -0.2249656 1.18365808 
         7 -0.2200195 2.39017316 
         8 -0.2150734 4.75552947 
         9 -0.2101273 9.32271020 
        10 -0.2051813 1.80080663 
        11 -0.2002352 3.42753445 
        12 -0.1952891 6.42828558 
        13 -0.1903430 1.18799975 
        14 -0.1853969 2.16349164 
       ...        ...        ... 
}, :extension=>[]}>

#<Nyaplot::Frame:0x007f6296962c10 @properties={:panes=>[#<Nyaplot::Plot:0x007f6296964f88 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f6296964588 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :title=>"Empirical"}, :data=>"b57c5534-500e-4bc8-bc59-0627dcbc1db5"}, @xrange=[-0.3168744682392347, 0.22159435209794193], @yrange=[0.0003937007874015748, 1.0]>, #<Nyaplot::Diagram:0x007f6296963930 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :title=>"Normal", :color=>"#44bb44"}, :data=>"669b6f48-111b-43af-af0e-08eca945e7b2"}, @xrange=[-0.3168744682392347, 0.22159435209794187], @yrange=[-2.220446049250313e-16, 1.0]>], :options=>{:xrange=>[-0.1, 0.1], :x_label=>"ln[P(t + 1d)/P(t)] of BK", :y_label=>"Cumulative probability", :legend=>true, :zoom=>true, :width=>800, :yrange=>[-2.220446049250313e-16, 1.0]}}>], :data=>{"b57c5534-500e-4bc8-bc59-0627dcbc1db5"=>
#<Daru::DataFrame:70030707507020 @name = b57c5534-500e-4bc8-bc59-0627dcbc1db5 @size = 2517>
                    x          y 
         0 -0.3168744 0.00039370 
         1 -0.1893145 0.00078740 
         2 -0.1725877 0.00118110 
         3 -0.1715250 0.00157480 
         4 -0.1558623 0.00196850 
         5 -0.1311873 0.00236220 
         6 -0.1204489 0.00275590 
         7 -0.1123763 0.00314960 
         8 -0.1116004 0.00354330 
         9 -0.1069879 0.00393700 
        10 -0.1058566 0.00433070 
        11 -0.1024881 0.00472440 
        12 -0.1017107 0.00511811 
        13 -0.1014299 0.00551181 
        14 -0.1007028 0.00590551 
       ...        ...        ... 
, "669b6f48-111b-43af-af0e-08eca945e7b2"=>
#<Daru::DataFrame:70030705010040 @name = 669b6f48-111b-43af-af0e-08eca945e7b2 @size = 100>
                    x          y 
         0 -0.3168744        0.0 
         1 -0.3114353        0.0 
         2 -0.3059963        0.0 
         3 -0.3005572        0.0 
         4 -0.2951181        0.0 
         5 -0.2896790        0.0 
         6 -0.2842399        0.0 
         7 -0.2788009        0.0 
         8 -0.2733618        0.0 
         9 -0.2679227        0.0 
        10 -0.2624836        0.0 
        11 -0.2570445        0.0 
        12 -0.2516055        0.0 
        13 -0.2461664        0.0 
        14 -0.2407273        0.0 
       ...        ...        ... 
}, :extension=>[]}>

#<Nyaplot::Frame:0x007f6297544f10 @properties={:panes=>[#<Nyaplot::Plot:0x007f6297547418 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f6297546860 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :title=>"Empirical"}, :data=>"b9073e3d-9e1f-4db3-81f7-9d0f584c3f59"}, @xrange=[-0.13503625982529693, 0.17908555905799614], @yrange=[0.0003937007874015748, 1.0]>, #<Nyaplot::Diagram:0x007f6297545c08 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :title=>"Normal", :color=>"#44bb44"}, :data=>"7f892ed2-4c86-4119-ae29-1d631149fe85"}, @xrange=[-0.13503625982529693, 0.1790855590579961], @yrange=[0.0, 1.0]>], :options=>{:xrange=>[-0.1, 0.1], :x_label=>"ln[P(t + 1d)/P(t)] of NIO", :y_label=>"Cumulative probability", :legend=>true, :zoom=>true, :width=>800, :yrange=>[0.0, 1.0]}}>], :data=>{"b9073e3d-9e1f-4db3-81f7-9d0f584c3f59"=>
#<Daru::DataFrame:70030702973660 @name = b9073e3d-9e1f-4db3-81f7-9d0f584c3f59 @size = 2364>
                    x          y 
         0 -0.1350362 0.00039370 
         1 -0.0657169 0.00078740 
         2 -0.0610876 0.00118110 
         3 -0.0549084 0.00157480 
         4 -0.0536800 0.00196850 
         5 -0.0522928 0.00236220 
         6 -0.0446725 0.00275590 
         7 -0.0403990 0.00314960 
         8 -0.0399543 0.00354330 
         9 -0.0388397 0.00393700 
        10 -0.0358514 0.00433070 
        11 -0.0358441 0.00472440 
        12 -0.0353637 0.00511811 
        13 -0.0353080 0.00551181 
        14 -0.0320149 0.00590551 
       ...        ...        ... 
, "7f892ed2-4c86-4119-ae29-1d631149fe85"=>
#<Daru::DataFrame:70030711322520 @name = 7f892ed2-4c86-4119-ae29-1d631149fe85 @size = 100>
                    x          y 
         0 -0.1350362        0.0 
         1 -0.1318633        0.0 
         2 -0.1286903        0.0 
         3 -0.1255174        0.0 
         4 -0.1223444        0.0 
         5 -0.1191715        0.0 
         6 -0.1159985        0.0 
         7 -0.1128256        0.0 
         8 -0.1096526        0.0 
         9 -0.1064797        0.0 
        10 -0.1033067        0.0 
        11 -0.1001338        0.0 
        12 -0.0969608        0.0 
        13 -0.0937879        0.0 
        14 -0.0906149        0.0 
       ...        ...        ... 
}, :extension=>[]}>

In [10]:
# ダウ平均の累積分布
dji = Daru::DataFrame.from_csv("DJI.csv")
dji_lr = get_log_return(dji)
plot_cumulative_distribution(dji_lr[:log_return], "ln[P(t + 1d)/P(t)] of DJI")

#<Nyaplot::Frame:0x007f6296619b30 @properties={:panes=>[#<Nyaplot::Plot:0x007f629661be30 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f629661b408 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :title=>"Empirical"}, :data=>"248729c3-ea79-4150-a50f-d559ca7430e4"}, @xrange=[-0.2563151096383328, 0.10508346149108455], @yrange=[0.0001278772378516624, 1.0]>, #<Nyaplot::Diagram:0x007f629661a850 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :title=>"Normal", :color=>"#44bb44"}, :data=>"4a719c75-30c3-474f-b9ed-66c24c5ec11c"}, @xrange=[-0.2563151096383328, 0.10508346149108455], @yrange=[-2.220446049250313e-16, 1.0000000000000002]>], :options=>{:xrange=>[-0.1, 0.1], :x_label=>"ln[P(t + 1d)/P(t)] of DJI", :y_label=>"Cumulative probability", :legend=>true, :zoom=>true, :width=>800, :yrange=>[-2.220446049250313e-16, 1.0000000000000002]}}>], :data=>{"248729c3-ea79-4150-a50f-d559ca7430e4"=>
#<Daru::DataFrame:70030694676020 @name = 248729c3-ea79-4150-a50f-d559ca7430e4 @size = 7804>
                    x          y 
         0 -0.2563151 0.00012787 
         1 -0.0838102 0.00025575 
         2 -0.0820051 0.00038363 
         3 -0.0801400 0.00051150 
         4 -0.0761592 0.00063938 
         5 -0.0745407 0.00076726 
         6 -0.0739624 0.00089514 
         7 -0.0723449 0.00102301 
         8 -0.0715548 0.00115089 
         9 -0.0709723 0.00127877 
        10 -0.0657819 0.00140664 
        11 -0.0586340 0.00153452 
        12 -0.0582171 0.00166240 
        13 -0.0572506 0.00179028 
        14 -0.0570611 0.00191815 
       ...        ...        ... 
, "4a719c75-30c3-474f-b9ed-66c24c5ec11c"=>
#<Daru::DataFrame:70030703303260 @name = 4a719c75-30c3-474f-b9ed-66c24c5ec11c @size = 100>
                    x          y 
         0 -0.2563151        0.0 
         1 -0.2526646        0.0 
         2 -0.2490141        0.0 
         3 -0.2453636        0.0 
         4 -0.2417131        0.0 
         5 -0.2380626        0.0 
         6 -0.2344121        0.0 
         7 -0.2307616        0.0 
         8 -0.2271111        0.0 
         9 -0.2234606        0.0 
        10 -0.2198102        0.0 
        11 -0.2161597        0.0 
        12 -0.2125092        0.0 
        13 -0.2088587        0.0 
        14 -0.2052082        0.0 
       ...        ...        ... 
}, :extension=>[]}>

## 標準偏差は取引量の平方根に比例する?

In [11]:
lr = log_returns[0]
# 対数収益率データに年・月・四半期を追加
df = Daru::DataFrame.new(date: lr[:date], volume: lr[:volume], log_return: lr[:log_return],
  year: lr[:date].map { |d| d.slice(/\A\d{4}/) },
  quarter: lr[:date].map { |d| d.slice(/\A\d{4}-/) + (d.slice(/\A\d{4}-(\d{2})/, 1).to_i / 4 + 1).to_s + "Q" },
  month: lr[:date].map { |d| d.slice(/\A\d{4}-\d{2}/) })

,date,log_return,month,quarter,volume,year
0,2006-01-04,0.009430610335608217,2006-01,2006-1Q,365900,2006
1,2006-01-05,-0.013081581897489276,2006-01,2006-1Q,90300,2006
2,2006-01-06,0.03097114664826428,2006-01,2006-1Q,642000,2006
3,2006-01-09,0.000708968480599987,2006-01,2006-1Q,283900,2006
4,2006-01-10,0.03001970624065062,2006-01,2006-1Q,575500,2006
5,2006-01-11,-0.013850636933898937,2006-01,2006-1Q,177300,2006
6,2006-01-12,-0.014045174703047527,2006-01,2006-1Q,146200,2006
7,2006-01-13,0.03543267499464958,2006-01,2006-1Q,169500,2006
8,2006-01-17,-0.04181793968332148,2006-01,2006-1Q,348200,2006
9,2006-01-18,-0.018678703974544426,2006-01,2006-1Q,217500,2006


In [12]:
def get_sqrt_volume_and_std_log_return(df, group)
  x = []
  y = []
  df.group_by(group).each_group { |g|
    x << Math.sqrt(g[:volume].sum)
    y << g[:log_return].std
  }
  Daru::DataFrame.new(sqrt_volume: x, std_log_return: y)
end

last_decade = df.where(df[:year].gteq("2011"))
year = get_sqrt_volume_and_std_log_return(df, :year)
quarter = get_sqrt_volume_and_std_log_return(df, :quarter)
month = get_sqrt_volume_and_std_log_return(df, :month)
plot = Nyaplot::Plot.new
plot.add_with_df(year, :scatter, :sqrt_volume, :std_log_return).configure do |d|
  d.title("Year")
end
plot.add_with_df(quarter, :scatter, :sqrt_volume, :std_log_return).configure do |d|
  d.title("Quarter")
  d.color("#44bb44")
end
plot.add_with_df(month, :scatter, :sqrt_volume, :std_log_return).configure do |d|
  d.title("Month")
  d.color("#dd8844")
end
plot.x_label("Square root of volume")
plot.y_label("Standard deviation of ln[P(t + 1d)/P(t)]")
plot.legend(true)
plot.show

#<Nyaplot::Frame:0x007f62969ac298 @properties={:panes=>[#<Nyaplot::Plot:0x007f62969aee58 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f62969aebd8 @properties={:type=>:scatter, :options=>{:x=>:sqrt_volume, :y=>:std_log_return, :title=>"Year"}, :data=>"78bc9982-f7eb-42b5-a2a2-39eb6e07eb2b"}, @xrange=[3037.054494078103, 12595.657982019042], @yrange=[0.01898184614643102, 0.07185216909161421]>, #<Nyaplot::Diagram:0x007f62969add00 @properties={:type=>:scatter, :options=>{:x=>:sqrt_volume, :y=>:std_log_return, :title=>"Quarter", :color=>"#44bb44"}, :data=>"9afd6293-c64b-40ef-bc47-f6187d852eb4"}, @xrange=[2456.0741031165976, 7946.546419671881], @yrange=[0.01676837543121871, 0.10633586172445177]>, #<Nyaplot::Diagram:0x007f62969ad030 @properties={:type=>:scatter, :options=>{:x=>:sqrt_volume, :y=>:std_log_return, :title=>"Month", :color=>"#dd8844"}, :data=>"ca9436c2-25e6-4112-82f2-b914d055759e"}, @xrange=[1317.459676802292, 5809.475019311125], @yrange=[0.009627970820079625, 0.10988288065637761]>], :options=>{:x_label=>"Square root of volume", :y_label=>"Standard deviation of ln[P(t + 1d)/P(t)]", :legend=>true, :zoom=>true, :width=>800, :xrange=>[1317.459676802292, 12595.657982019042], :yrange=>[0.009627970820079625, 0.10988288065637761]}}>], :data=>{"78bc9982-f7eb-42b5-a2a2-39eb6e07eb2b"=>
#<Daru::DataFrame:70030712444220 @name = 78bc9982-f7eb-42b5-a2a2-39eb6e07eb2b @size = 11>
           sqrt_volum std_log_re 
         0 9035.91721 0.01898184 
         1 11399.1973 0.02612139 
         2 12198.9794 0.05923967 
         3 10137.9139 0.05519272 
         4 9715.35897 0.03737944 
         5 9382.31847 0.03515863 
         6 12595.6579 0.04685913 
         7 11485.0032 0.03109522 
         8 10576.7906 0.03426600 
         9 9125.91913 0.03308707 
        10 3037.05449 0.07185216 
, "9afd6293-c64b-40ef-bc47-f6187d852eb4"=>
#<Daru::DataFrame:70030681137880 @name = 9afd6293-c64b-40ef-bc47-f6187d852eb4 @size = 41>
           sqrt_volum std_log_re 
         0 4420.41853 0.01719766 
         1 5432.07142 0.01756827 
         2 5154.41558 0.02092368 
         3 2456.07410 0.02194384 
         4 4623.69981 0.01676837 
         5 6095.83464 0.02033729 
         6 7878.00736 0.03384767 
         7 3056.28859 0.03265496 
         8 6494.89029 0.04882284 
         9 7105.83562 0.03378819 
        10 6884.38813 0.06931908 
        11 2956.99171 0.10633586 
        12 4500.65550 0.08246593 
        13 6243.81293 0.05071422 
        14 6042.67324 0.03819338 
       ...        ...        ... 
, "ca9436c2-25e6-4112-82f2-b914d055759e"=>
#<Daru::DataFrame:70030705169320 @name = ca9436c2-25e6-4112-82f2-b914d055759e @size = 122>
           sqrt_volum std_log_re 
         0 2385.51881 0.02311638 
         1 2278.77159 0.00962797 
         2 2942.21005 0.01627789 
         3 2084.03454 0.01491641 
         4 2962.98835 0.01977831 
         5 3053.45705 0.01976540 
         6 2657.31067 0.01486693 
         7 2627.56541 0.02398749 
         8 2186.34398 0.02336291 
         9 2291.13508 0.01695526 
        10 3103.94909 0.01878565 
        11 2456.07410 0.02194384 
        12 2727.85630 0.01232400 
        13 2412.88209 0.01699606 
        14 2848.75411 0.01948683 
       ...        ...        ... 
}, :extension=>[]}>

In [13]:
month.where(month[:std_log_return].gt(0.10))

,sqrt_volume,std_log_return
35,2956.9917145639756,0.10633586172445177
38,3031.1219045099456,0.10988288065637761
121,1551.6120649182901,0.10974812021248168
